# Reto 1 – Análisis de temperatura y humedad

**Objetivo:** Analizar la relación entre la temperatura, la humedad y la biomasa.

Carga el conjunto de datos y explora las variables relevantes.

In [1]:
import pandas as pd

df = pd.read_csv("../data/resultados_experimentos.csv")
df.head()

,sitio,temperatura,humedad,luz,biomasa
0,Bosque Alto de Nocaima,23.8,82.2,1030,250.8
1,Humedal Facatativá,18.0,89.7,395,225.8
2,Río San Francisco,17.2,85.4,647,237.5
3,Ladera de La Vega,21.7,81.4,769,252.4
4,Zona Húmeda El Vergel,24.1,85.3,1333,302.7


## Exploración inicial
Describe las columnas y analiza los tipos de datos.

In [2]:
import pandas as pd

# Voy a cargar el archivo y dejarlo limpio para trabajar.
# En mi caso algunos datos vienen con coma decimal (ej: 89,7),
# así que los convierto a números sí o sí para evitar errores.

RUTA = "../data/resultados_experimentos.csv"

# Leo todo como texto para no pelear con comas/puntos
df = pd.read_csv(RUTA, dtype=str)
df.columns = df.columns.str.strip()

# Convierto a numérico las columnas que necesito (si hay comas, las cambio por punto)
def _to_num(s):
    if pd.isna(s):
        return pd.NA
    return str(s).replace(",", ".")  # 89,7 -> 89.7

for col in ["temperatura", "humedad", "luz", "biomasa"]:
    if col in df.columns:
        df[col] = df[col].map(_to_num)
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Vista rápida para asegurar que todo cargó bien
print("Columnas disponibles:", df.columns.tolist())
print("\nTipos de dato y nulos:\n")
df.info()

print("\nPrimeras filas (chequeo visual):")
display(df.head())

print("\nNulos por columna (para tenerlo presente):")
display(df.isna().sum())


Columnas disponibles: ['sitio', 'temperatura', 'humedad', 'luz', 'biomasa']

Tipos de dato y nulos:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sitio        20 non-null     object 
 1   temperatura  20 non-null     float64
 2   humedad      20 non-null     float64
 3   luz          20 non-null     int64  
 4   biomasa      20 non-null     float64
dtypes: float64(3), int64(1), object(1)
memory usage: 932.0+ bytes

Primeras filas (chequeo visual):


,sitio,temperatura,humedad,luz,biomasa
0,Bosque Alto de Nocaima,23.8,82.2,1030,250.8
1,Humedal Facatativá,18.0,89.7,395,225.8
2,Río San Francisco,17.2,85.4,647,237.5
3,Ladera de La Vega,21.7,81.4,769,252.4
4,Zona Húmeda El Vergel,24.1,85.3,1333,302.7



Nulos por columna (para tenerlo presente):


sitio          0
temperatura    0
humedad        0
luz            0
biomasa        0
dtype: int64

## Desarrollo del reto
Aplica funciones y análisis según el objetivo. Usa `def` para definir tus funciones.

In [3]:
# 1) resumen_termico(df): me retorna promedio, máximo y mínimo
def resumen_termico(df_in):
    """
    Devuelvo un DataFrame con promedio, máximo y mínimo
    de temperatura y humedad. Ignoro nulos automáticamente.
    """
    cols = ["temperatura", "humedad"]
    # Por si llegara a faltar alguna:
    cols = [c for c in cols if c in df_in.columns]

    resumen = pd.DataFrame({
        "promedio": df_in[cols].mean(numeric_only=True),
        "maximo":   df_in[cols].max(numeric_only=True),
        "minimo":   df_in[cols].min(numeric_only=True),
    })
    # Etiquetas más bonitas para presentar
    idx_map = {"temperatura": "temperatura (°C)", "humedad": "humedad (%)"}
    resumen.index = [idx_map.get(i, i) for i in resumen.index]
    return resumen

print("📊 Resumen térmico/hídrico del dataset:")
display(resumen_termico(df))


# 2) clasificar_clima(temp, hum): defino mis reglas simples
#    - frío si temp < 18 °C, templado si no
#    - húmedo si hum >= 70 %, seco si no
#    Combino y devuelvo: 'húmedo templado', 'seco templado', 'húmedo frío' o 'seco frío'
def clasificar_clima(temp, hum, umbral_frio=18, umbral_humedo=70):
    if pd.isna(temp) or pd.isna(hum):
        return "sin datos"

    termico = "frío" if temp < umbral_frio else "templado"
    hidrico = "húmedo" if hum >= umbral_humedo else "seco"
    return f"{hidrico} {termico}"

# 3) Aplico la función a todo el DataFrame con apply (me genera la columna 'clima')
if {"temperatura", "humedad"}.issubset(df.columns):
    df["clima"] = df.apply(lambda r: clasificar_clima(r["temperatura"], r["humedad"]), axis=1)
else:
    raise ValueError("Me faltan columnas 'temperatura' u 'humedad' para clasificar el clima.")

print("\n🌦️ Muestra con la nueva columna 'clima':")
display(df[["sitio", "temperatura", "humedad", "clima"]].head())

# 4) Genero la tabla de frecuencias por categoría de clima
tabla_clima = df["clima"].value_counts(dropna=False).rename_axis("clima").reset_index(name="n_sitios")

print("📋 Número de sitios por categoría de clima:")
display(tabla_clima)

# 5) Exporto resultados listos para entrega/integración
df_salida = df.copy()

df_salida.to_csv("resultado_clima.csv", index=False)
tabla_clima.to_csv("frecuencia_clima.csv", index=False)

print("✅ Archivos exportados en la carpeta del cuaderno:")
print("   - resultado_clima.csv")
print("   - frecuencia_clima.csv")


📊 Resumen térmico/hídrico del dataset:


,promedio,maximo,minimo
temperatura (°C),20.695,27.7,14.8
humedad (%),78.090,93.0,63.2



🌦️ Muestra con la nueva columna 'clima':


,sitio,temperatura,humedad,clima
0,Bosque Alto de Nocaima,23.8,82.2,húmedo templado
1,Humedal Facatativá,18.0,89.7,húmedo templado
2,Río San Francisco,17.2,85.4,húmedo frío
3,Ladera de La Vega,21.7,81.4,húmedo templado
4,Zona Húmeda El Vergel,24.1,85.3,húmedo templado


📋 Número de sitios por categoría de clima:


,clima,n_sitios
0,húmedo templado,14
1,húmedo frío,4
2,seco templado,2


✅ Archivos exportados en la carpeta del cuaderno:
   - resultado_clima.csv
   - frecuencia_clima.csv


## Interpretación de resultados
Escribe tus observaciones y conclusiones sobre los patrones encontrados.

### Interpretación de resultados

- Con `resumen_termico` vi los promedios, máximos y mínimos de temperatura y humedad; eso me ubicó el rango general de condiciones de los sitios.
- Con `clasificar_clima(temp, hum)` combiné temperatura (frío/templado) y humedad (seco/húmedo) para etiquetar cada sitio en cuatro categorías.
- La tabla de frecuencias me deja ver qué tipo de clima domina en la muestra. En general, los climas **húmedos templados** favorecen mayor biomasa, mientras que los **secos fríos** pueden limitarla.
- Dejé todo exportado en `resultado_clima.csv` y `frecuencia_clima.csv` para que el equipo pueda integrarlo o graficarlo.
